In [1]:
import torch
from transformers import Data2VecTextForMaskedLM, AutoTokenizer
from PIL import Image
import requests

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [53]:
version = "facebook/data2vec-text-base"
text = "The capital of France is <mask>."

# AutoTokenizer

In [54]:
tokenizer: AutoTokenizer = AutoTokenizer.from_pretrained(version)
tokenizer

RobertaTokenizerFast(name_or_path='facebook/data2vec-text-base', vocab_size=50265, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)}, clean_up_tokenization_spaces=True)

## special ids and tokens

In [55]:
print(tokenizer.all_special_ids)
print(tokenizer.all_special_tokens)

[0, 2, 3, 1, 50264]
['<s>', '</s>', '<unk>', '<pad>', '<mask>']


In [56]:
tokenizer.mask_token_id

50264

## processor

In [57]:
inputs = tokenizer(
    text = text,                # 可以为列表或单个string
    return_tensors = "pt",      # 返回数据格式 np pt tf jax
    padding = True,             # 填充方式选择 [True, 'longest', 'max_length', 'do_not_pad']
    # max_length = max_length,  # 如果使用max_length要将padding设置为 "max_length"
    add_special_tokens = True,  # text添加特殊key
).to(device, torch.float16)
inputs

{'input_ids': tensor([[    0,   133,   812,     9,  1470,    16, 50264,     4,     2]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [58]:
inputs["input_ids"]

tensor([[    0,   133,   812,     9,  1470,    16, 50264,     4,     2]],
       device='cuda:0')

# Data2VecTextForMaskedLM

data2vec Model with a language modeling head on top. 

In [59]:
model: Data2VecTextForMaskedLM = Data2VecTextForMaskedLM.from_pretrained(version, torch_dtype=torch.float16).to(device)
model

Data2VecTextForMaskedLM(
  (data2vec_text): Data2VecTextModel(
    (embeddings): Data2VecTextForTextEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): Data2VecTextEncoder(
      (layer): ModuleList(
        (0-11): 12 x Data2VecTextLayer(
          (attention): Data2VecTextAttention(
            (self): Data2VecTextSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): Data2VecTextSelfOutput(
              (dense): Linear(in_features=768, out

In [60]:
model.eval()
with torch.inference_mode():
    outputs = model(**inputs)
outputs

MaskedLMOutput(loss=None, logits=tensor([[[ 0.2491,  0.0000,  1.4818,  ..., -0.0457,  0.1199,  0.1965],
         [ 2.2702,  0.0000,  1.7460,  ..., -0.3340,  0.0366, -0.5271],
         [ 2.2095,  0.0000,  1.7296,  ..., -0.3283,  0.0243, -0.4372],
         ...,
         [ 0.2935,  0.0000,  0.4662,  ...,  0.0253,  0.0524,  0.3958],
         [ 0.2407,  0.0000,  1.4914,  ..., -0.0457,  0.1169,  0.2166],
         [-1.4884,  0.0000, -0.5424,  ...,  0.1615, -0.3163,  0.3661]]],
       device='cuda:0'), hidden_states=None, attentions=None)

In [61]:
logits = outputs.logits
print(logits.shape)
print(logits)

torch.Size([1, 9, 50265])
tensor([[[ 0.2491,  0.0000,  1.4818,  ..., -0.0457,  0.1199,  0.1965],
         [ 2.2702,  0.0000,  1.7460,  ..., -0.3340,  0.0366, -0.5271],
         [ 2.2095,  0.0000,  1.7296,  ..., -0.3283,  0.0243, -0.4372],
         ...,
         [ 0.2935,  0.0000,  0.4662,  ...,  0.0253,  0.0524,  0.3958],
         [ 0.2407,  0.0000,  1.4914,  ..., -0.0457,  0.1169,  0.2166],
         [-1.4884,  0.0000, -0.5424,  ...,  0.1615, -0.3163,  0.3661]]],
       device='cuda:0')


In [62]:
# 获取maskid
mask_token_index = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
mask_token_index

tensor([6], device='cuda:0')

In [63]:
predicted_token_id = logits[0, mask_token_index].argmax(axis=-1)
predicted_token_id

tensor([2560], device='cuda:0')

In [64]:
tokenizer.decode(predicted_token_id)

'la'

In [65]:
text.replace("<mask>", tokenizer.decode(predicted_token_id))

'The capital of France is la.'